# Тестовое задание в команду Data Science IDP

## Формулировка задания

**Написать решение по извлечению сущностей из документов (новостных текстов)**

**Датасет** -- семпл текстов с аннотациями русских новостей (источник: https://bsnlp.cs.helsinki.fi/bsnlp-2019/shared_task.html)

**Интересующие сущности:**
- *PER* - persons (личности)
- *ORG* - organizations (организации)
- *LOC* - locations (места)
- *EVT* - events (события)
- *PRO* - products (продукты и документы)

Достаточно использовать *9 документов* про *брекзит* из семпла

## Задание 1

**Описание задачи с точки зрения NLP**

Это задача извлечения именованных сущностей из текст -- *Named Entity Recognition (NER)*. В данном случае мы занимаемся последовательной разметкой, классифицируя каждый токен как PER, ORG, LOC, EVT, PRO или не являющийся сущностью

**Классические методы решения:**
- Статистические методы: *Условные случайные поля (CRF)*, *Марковская модель максимальной энтропии (MEMM)*
- Рекуррентные нейронные сети: *LSTM*, *BiLSTM* (часто с CRF слоем)
- Энкодерные трансформеры: *BERT-like*

**LLM методы:**
- *Zero-shot/Few-shot* подходы (GPT, в т.ч. *GigaChat*)
- Промты с инструкциями для GPT-моделей
- *Fine-tuning* на обучающей выборке
- *Chain-of-Thought (CoT)* промпты для размышления перед определением сущности

**Метрики качества:**
- *Precision, Recall и F1-score* для найденных сущностей (верные границы), **без учета типа сущности**
- *Precision, Recall и F1-score* для **точного совпадения** (границы + тип сущности)
- Эти же метрики с *частичным совпадением границ*
- Эти же метрики для каждого отдельного класса (для выявления, с какими типами сущностей у модели проблемы). Macro-F1

- Можно добавить *accuracy* лемматизации каждой верно найденной сущности (для задачи из оригинального соревнования)
- *Inference time* (в данном случае зависит от провайдера GigaChat)

## Задание 2

In [18]:
import pandas as pd

import os
import pathlib
import re

from typing import List, Tuple

### Чтение датасета в pandas DataFrame с обязательными колонками "document_id", "document_text", "entity", "gold_answer"

#### Функция для извлечения данных и формирования DataFrame

In [19]:
def read_bsnlp_data(data_dir: str='sample_pl_cs_ru_bg', lang: str="ru") -> pd.DataFrame:
    """
    Считывание данных соревнования из папок raw/ru и annotated/ru и преобразует в
    требуемый формат pd.DataFrame: ["document_id", "document_text", "entity", "gold_answer"]
    """
    raw_dir = pathlib.Path(data_dir) / 'raw' / lang
    annotated_dir = pathlib.Path(data_dir) / 'annotated' / lang

    documents_desc: List[dict] = []

    # Raw файлы
    for raw_file in raw_dir.glob('*.txt'):
        with open(raw_file, encoding="utf-8") as f:
            lines: List[str] = f.readlines()

        assert len(lines) >= 5, f"File \"{str(raw_file)}\" contains less than 5 lines"

        document_id = lines[0].strip()
        language = lines[1].strip()
        date = lines[2].strip()
        url = lines[3].strip()
        title = lines[4].strip()
        content = ' '.join([line.strip() for line in lines[5:] if line.strip()])
        document_text = title + ' ' + content

        # Ищем соответствующий аннотированный файл
        annotation_file = annotated_dir / f"{raw_file.stem}.out"

        assert annotation_file.exists(), \
            f"Annotation file \"{str(annotation_file)}\" corresponding to raw file \"{str(raw_file)}\" does not exist"
        
        # Читаем аннотации
        with open(annotation_file, encoding='utf-8') as f:
            annotation_lines = f.readlines()

        assert len(annotation_lines) > 0, f"Annotation file \"{str(annotation_file)}\" is empty"
        annotation_doc_id = annotation_lines[0].strip()

        assert document_id == annotation_doc_id, "Raw and Annotated doc ids do not match: " \
                                                f"{document_id}, {annotation_doc_id}"

        entities_found = []

        for line in annotation_lines[1:]:
            line = line.strip()
            if line:
                columns = line.split('\t')
                assert len(columns) == 4

                entity_text, entity_text_lemmatized, entity_type, entity_id = columns

                entities_found.append( (entity_text, entity_type) )
        
        for entity_text, entity_type in entities_found:
            documents_desc.append({
                'document_id': document_id,
                'document_text': document_text,
                'entity': entity_text,
                'gold_answer': entity_type
            })
        if not entities_found:
            documents_desc.append({
                'document_id': document_id,
                'document_text': document_text,
                'entity': None,
                'gold_answer': None
            })
        
    return pd.DataFrame(documents_desc)


#### Просмотр данных

In [21]:
df = read_bsnlp_data()
df.head()

,document_id,document_text,entity,gold_answer
0,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Brexit,EVT
1,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Альбиона,LOC
2,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Альбионе,LOC
3,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Борис Джонсон,PER
4,ru-10,Тереза Мэй рассчитывает усидеть в седле до зав...,Британии,LOC
